Machine Learning - Homework 1 (due Sep. 13)

Problem 1: Legal reasoning (from Murphy 2.2).

Suppose a crime has been committed. Blood is found at the scene for which there is no innocent explanation. It is of a type which is present in 1% of the population.  The defendant is known to have this rare blood type.

a. The prosecutor claims: “There is a 1% chance that the defendant would have the crime blood type if he
were innocent. Thus there is a 99% chance that he guilty”. This is known as the prosecutor’s fallacy.
What is wrong with this argument?

The 1% chance is actually the conditional probability that a random person would match that blood type given that they're innocent (P(B=1|G=0)). The prosecutor has swapped the uncertainty and the condition around, making the 1% chance the likelihood that the person is innocent given they have the blood type (P(G=0|B=1)), but those two conditional probabilities are not equal. To get the correct answer for P(G=0|B=1), along with the P(B=1|G=0) = 1% (the likelihood), they must also take into account the prior, which is the probability of the defendant being the one person in the city who did commit the act. Because there are 800,000 people in the city, the 1/800000 probability of guilt makes his innocence extremely likely. 

b. The defender claims: “The crime occurred in a city of 800,000 people. The blood type would be
found in approximately 8000 people. The evidence has provided a probability of just 1 in 8000 that
the defendant is guilty, and thus has no relevance.” This is known as the defender’s fallacy. What is
wrong with this argument (HINT: What happens to the prior in this case if there is *other* evidence presented)?

This probability is unconditional, only looking at P(G), i.e. the chance the defendant is the one in 8000 who committed the crime. Instead, this should be a conditional probability that also includes the prior probability of the defendant's guilt in the calculation. 

c. Suppose that forensic analysis tells us that that the blood test has 98% sensitivity (true positive rate) and a 1% false positive rate.  Given the information presented in the above two questions, determine the posterior probability the the defendent is guilty, given that the defendent's blood type matches that found at the crime scene *and* that the defendent was one of only 5 people with access to the crime scene *and* that there is no other evidence.  

Letting G = event that he's guilty and B = event that the blood matches;
P(G=1) = 1/5, P(G=0) = 4/5, P(B=1|G=1) = .98, and P(B=1|G=0) = .01. Because we now have the information about only 5 people having access to the crime scene and the statistics about the blood test accuracy, the information given prior about the frequency of the blood type and the number of people in the city is now unnecessary. Thus:
$$ P(G=\mathrm{1}|B=\mathrm{1}) = \frac{P(B=\mathrm{1}|G=\mathrm{1})P(G=\mathrm{1})}{P(B=\mathrm{1})} $$

$$ = \frac{P(B=\mathrm{1}|G=\mathrm{1})P(G=\mathrm{1})}{P(B=\mathrm{1}|G=\mathrm{1})P(G=\mathrm{1})+P(B=\mathrm{1}|G=\mathrm{0})P(G=\mathrm{0})} $$

$$ P(G=\mathrm{1}|B=\mathrm{1}) = \frac{(0.98)(0.2)}{(0.98)(0.2)+(0.01)(0.8)} = 0.96078431372 \approx 96\% $$

Problem 2: Naive Bayes.

One use of the naive Bayes classifier, which is still in practical use today, is as a spam filter.  Consider the corpus of text messages packaged with this homework, which are each labelled as either 'spam' or 'ham'.  In this case, naive Bayes utilizes a Bernoulli model that quantifies the probability of a given word given that the message is either spam or ham.  For example, investigating the text messages here, we find that the word *draw* shows up in spam 27 times, yet in ham only 5.  Thus, we have that
$$ P(X=\mathrm{draw}|Y=\mathrm{ham}) = \frac{5}{5+27}. $$

While this is not particularly strong evidence on its own, we can create a powerful classifier by using the naive assumption in conjunction with all the words in a given message:
$$ P(Y=\mathrm{ham}|X=x) \propto P(Y=\mathrm{ham}) \prod_{i=1}^n P(X_i=x_i|Y=\mathrm{ham}), $$
$$ P(Y=\mathrm{spam}|X=x) \propto P(Y=\mathrm{spam}) \prod_{i=1}^n P(X_i=x_i|Y=\mathrm{spam}), $$
where $x_i$ are the words in a given message. 

Your task is to write such a classifier.  I have taken the somewhat tedious step of parsing the data for you, yielding the variables *word_dictionary*, which contains the ham and spam counts for each word, as well as *training_labels*, which provides the spam/ham labels for each text message.  I have also parsed a set of test data: *test_messages* is a list, each entry containing another list of the words in the text message, as well as *test_labels* which contains the spam/ham label for each message.

In [1]:
import numpy as np

# Maps from 'ham' or 'spam' strings to zero or one
def mapper(s):
    if s=='spam':
        return 0
    else:
        return 1

# Read in the text file
f = open('SMSSpamCollection','r')
lines = f.readlines()

# Break out the test data
test_lines = lines[:len(lines)//5]
lines = lines[len(lines)//5:]

# Instantiate the frequency dictionary and an array to
# record whether the line is ham or spam
word_dictionary = {}
training_labels = np.zeros(len(lines),dtype=int)

# Loop over all the training messages
for i,l in enumerate(lines):
    # Split into words
    l = l.lower().split()
    # Record the special first word which always ham or spam
    if l[0]=='ham':
        training_labels[i] = 1
    # For each word in the message, record whether the message was ham or spam
    for w in l[1:]:
        # If we've never seen the word before, add a new dictionary entry
        if w not in word_dictionary:
            word_dictionary[w] = [1,1]
        word_dictionary[w][mapper(l[0])] += 1
        
# Loop over the test messages
test_labels = np.zeros(len(test_lines),dtype=int)
test_messages = []
for i,l in enumerate(test_lines):
    l = l.lower().split()
    if l[0]=='ham':
        test_labels[i] = 1
    test_messages.append(l)

Below, I have provided code skeletons.  Your job is to make the code skeletons into an operational naive Bayes spam detector.  (you may discard these skeletons if you would prefer to code this from scratch).  Note that lines where you will need to change the code are marked with a '#!'.

Your first task is train the model:  

In [7]:
#What is the prior P(Y=ham) ?
ham_prior = np.sum(training_labels)/len(training_labels)  # !

# What are the class probabilities P(X=word|Y=ham) for each word?
ham_likelihood = {}
for key,val in word_dictionary.items():
    ham_likelihood[key] = word_dictionary[key][1]/(word_dictionary[key][0] + word_dictionary[key][1])

0.8701793721973095


Your next task is to make predictions on a set of test examples which were held back from the training procedure (see *test_messages* variable).  For each of these messages, compute the ham and spam probabilities.

In [5]:
# Where to hold the ham and spam posteriors
posteriors = np.zeros((len(test_lines),2))

# Loop over all the messages in the test set
for i,m in enumerate(test_messages):
    spam_prior = 1 - ham_prior
    posterior_ham = 1.0 * ham_prior
    posterior_spam = 1.0 * spam_prior
    #! Don't forget to include the prior!
    
    # Loop over all the words in each message
    for w in m:
        # #! What is the purpose of this try/except handler? It will skip over all the words that don't exist in the training data
        try:
            posterior_ham *= (ham_likelihood[w]) 
            posterior_spam *= (1-ham_likelihood[w])
        except KeyError:
            pass
    
    # Notice the normalization factor (denominator) 
    # to turn these into proper probabilities!
    posteriors[i,0] = posterior_spam/(posterior_spam + posterior_ham)
    posteriors[i,1] = posterior_ham/(posterior_spam + posterior_ham)

Finally, make a ham/spam prediction based on your posterior probabilities.  Compare these to the labels contained in test_labels.  Report the accuracy of your classifier as percentage correct.

In [10]:
total = 0
correct = 0
pctCorrect = 0
# Instantiate the array for space to record predictions
predictions= np.zeros(len(test_lines),dtype=int)

# checks which posterior is larger and then puts the proper one in the array
for i,value in enumerate(predictions):
    if(posteriors[i,0] >= posteriors[i,1]):
        predictions[i] = 0
    else:
        predictions[i] = 1

    #checks to see if that prediction is correct and increments # correct if so. Then increments total whether or not correct
    if(predictions[i] == test_labels[i]):
        correct += 1
    total +=1 

#converts the total into a percentage
pctCorrect = int((correct/total) * 100)

#outputs accuracy
print("This classifier has a " + str(pctCorrect) + "% accuracy rate.")

This classifier has a 93% accuracy rate.
